In [1]:
from pyspark.sql import SparkSession
from config import *
spark = SparkSession. \
  builder. \
  master("local[*]"). \
  appName("main"). \
  getOrCreate()

df_swiggy = spark. \
  read.format("csv"). \
  option("header", "true"). \
  load("../data/swiggy.csv")

df_zomato = spark. \
  read.format("csv"). \
  option("header", "true"). \
  load("../data/zomato.csv")
# empty_df = spark.createDataFrame([], schema)



25/01/16 16:59:49 WARN Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 192.168.100.201 instead (on interface wlp0s20f3)
25/01/16 16:59:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 16:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
for old_name, new_name in rename_swiggy.items():
    df_swiggy = df_swiggy.withColumnRenamed(old_name, new_name)
df_swiggy = df_swiggy.drop("ID")
for old_name, new_name in rename_zomato.items():
    df_zomato = df_zomato.withColumnRenamed(old_name, new_name)
df_zomato = df_zomato.drop("Unnamed: 0")
df_swiggy.show(1)
df_zomato.show(1)

+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+
|Area swiggy|Restaurant city swiggy|Restaurant swiggy|Price swiggy|Ratings swiggy|Total ratings|    Food type swiggy|Customer address swiggy|Delivery time swiggy|
+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+
|Koramangala|             Bangalore|      Tandoor Hut|       300.0|           4.4|          100|Biryani,Chinese,N...|              5Th Block|                  59|
+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+
only showing top 1 row

+---+-----------------+---------------+--------------+--------------------+------------+------------+-------------+----------------+-----------+--------------+
|

25/01/16 16:59:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
 Schema: _c0, restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
Expected: _c0 but found: 
CSV file: file:///home/vtzy/Documents/code/warehouse/data/zomato.csv


In [6]:

merged_df = df_swiggy.join(df_zomato, df_swiggy["Restaurant swiggy"] == df_zomato["Restaurant zomato"], "inner")

# Show the result
merged_df.show(2)
merged_df = merged_df.drop("Restaurant zomato")
merged_df.show(2)
merged_df = merged_df.withColumnRenamed("Restaurant swiggy", "Restaurant")
merged_df.show(2)

25/01/16 17:02:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
 Schema: _c0, restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
Expected: _c0 but found: 
CSV file: file:///home/vtzy/Documents/code/warehouse/data/zomato.csv


+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+----+-----------------+---------------+--------------+--------------------+------------+------------+-------------+--------------------+-----------------+-----------------+
|Area swiggy|Restaurant city swiggy|Restaurant swiggy|Price swiggy|Ratings swiggy|Total ratings|    Food type swiggy|Customer address swiggy|Delivery time swiggy| _c0|Restaurant zomato|restaurant type|Ratings zomato|Total ratings zomato|Price zomato|online_order|table booking|    Food type zomato|      Area zomato|   Address zomato|
+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+----+-----------------+---------------+--------------+--------------------+------------+------------+-------------+--------------------+-----------------+--------------

25/01/16 17:02:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
 Schema: _c0, restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
Expected: _c0 but found: 
CSV file: file:///home/vtzy/Documents/code/warehouse/data/zomato.csv
25/01/16 17:02:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
 Schema: _c0, restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
Expected: _c0 but found: 
CSV file: file:///home/vtzy/Documents/code/wa

+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+----+---------------+--------------+--------------------+------------+------------+-------------+--------------------+-----------------+-----------------+
|Area swiggy|Restaurant city swiggy|Restaurant swiggy|Price swiggy|Ratings swiggy|Total ratings|    Food type swiggy|Customer address swiggy|Delivery time swiggy| _c0|restaurant type|Ratings zomato|Total ratings zomato|Price zomato|online_order|table booking|    Food type zomato|      Area zomato|   Address zomato|
+-----------+----------------------+-----------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+----+---------------+--------------+--------------------+------------+------------+-------------+--------------------+-----------------+-----------------+
|Koramangala|             Bangalore|      Tandoor

In [9]:
merged_df.show(2)

+-----------+----------------------+-------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+----+---------------+--------------+--------------------+------------+------------+-------------+--------------------+-----------------+-----------------+
|Area swiggy|Restaurant city swiggy|   Restaurant|Price swiggy|Ratings swiggy|Total ratings|    Food type swiggy|Customer address swiggy|Delivery time swiggy| _c0|restaurant type|Ratings zomato|Total ratings zomato|Price zomato|online_order|table booking|    Food type zomato|      Area zomato|   Address zomato|
+-----------+----------------------+-------------+------------+--------------+-------------+--------------------+-----------------------+--------------------+----+---------------+--------------+--------------------+------------+------------+-------------+--------------------+-----------------+-----------------+
|Koramangala|             Bangalore|  Tandoor Hut|       300.

25/01/16 17:11:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
 Schema: _c0, restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
Expected: _c0 but found: 
CSV file: file:///home/vtzy/Documents/code/warehouse/data/zomato.csv


In [15]:
merged_df.write \
    .format("csv") \
    .mode("append") \
    .option("path", "/home/vtzy/Documents/code/warehouse/data") \
    .option("header", "true") \
    .save()



25/01/16 17:19:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
 Schema: _c0, restaurant name, restaurant type, rate (out of 5), num of ratings, avg cost (two people), online_order, table booking, cuisines type, area, local address
Expected: _c0 but found: 
CSV file: file:///home/vtzy/Documents/code/warehouse/data/zomato.csv
